# **Training Sockeye model for Russian to Chukchi translation**

In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 1

## **Preparing Data**

In [ ]:
import pandas as pd

df = pd.read_csv('/content/ckt-ru_filtered.csv', delimiter=";")

In [ ]:
df

,ru,ckt
0,Кит и олень,Ръэв ынкъам ӄораны
1,"Олень шел по берегу моря, и кит крикнул:","Аӈӄачормэпы нылеӄин ӄораӈы, ръэв-ым аӈӄайпы ны..."
2,давай сразимся.,- Китаӄун мынтэйкэвмык.
3,Кто сильнее - ты или я?,Мэӈин эплеэн армаӈ вальын!
4,- Хорошо!,- Ээ!
...,...,...
69220,Фигуры уже были убраны с шахматной доски.,Чьумэт энмэч чьоматкоёлгэпы алва гэнлелинэт.
69221,Шахматы.,Чьумэт
69222,"о, это ещё шахматы!","А'мын анны, лымӈэ ынӄэнат чьумэт!"
69223,"заниматься спортом, тренироваться",Илюльэтык


In [ ]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['ckt', 'ru'],
    num_rows: 67258
})

In [ ]:
shf_dataset = ds.shuffle(seed=42)
split_dataset = shf_dataset.train_test_split(test_size=0.01)

In [ ]:
!mkdir '/content/data'
!mkdir '/content/data/train'
!mkdir '/content/data/dev'

In [ ]:
def writelines(list, filename):
  with open(filename, 'w', encoding='utf-8') as file:
    for item in list:
      file.write('%s\n' % item)

ckt_train = [l['ckt'] for l in split_dataset['train']]
rus_train = [l['ru'] for l in split_dataset['train']]
ckt_dev = [l['ckt'] for l in split_dataset['test']]
rus_dev = [l['ru'] for l in split_dataset['test']]

writelines(ckt_train, '/content/data/train/train.ckt.txt')
writelines(rus_train, '/content/data/train/train.rus.txt')
writelines(ckt_dev, '/content/data/dev/dev.ckt.txt')
writelines(rus_dev, '/content/data/dev/dev.rus.txt')

## **Loading and Training the Model**

In [ ]:
!pip install sockeye

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: torch
    Found existing i

In [ ]:
!git clone https://github.com/rsennrich/subword-nmt.git

Cloning into 'subword-nmt'...
remote: Enumerating objects: 622, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 622 (delta 25), reused 31 (delta 16), pack-reused 576 (from 1)
Receiving objects: 100% (622/622), 261.27 KiB | 16.33 MiB/s, done.
Resolving deltas: 100% (374/374), done.


In [ ]:
src = 'rus'
tgt = 'ckt'
name = '69K'

In [ ]:
!python subword-nmt/subword_nmt/learn_joint_bpe_and_vocab.py --input /content/data/train/train.{src}.txt /content/data/train/train.{tgt}.txt -s 10000 -o /content/data/bpe.codes --write-vocabulary /content/data/bpe.vocab.{src} /content/data/bpe.vocab.{tgt}

100% 10000/10000 [00:30<00:00, 322.66it/s]


In [ ]:
!python subword-nmt/subword_nmt/apply_bpe.py -c /content/data/bpe.codes --vocabulary /content/data/bpe.vocab.{src} --vocabulary-threshold 50 < /content/data/train/train.{src}.txt > /content/data/train/train.{src}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c /content/data/bpe.codes --vocabulary /content/data/bpe.vocab.{tgt} --vocabulary-threshold 50 < /content/data/train/train.{tgt}.txt > /content/data/train/train.{tgt}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c /content/data/bpe.codes --vocabulary /content/data/bpe.vocab.{src} --vocabulary-threshold 50 < /content/data/dev/dev.{src}.txt > /content/data/dev/dev.{src}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c /content/data/bpe.codes --vocabulary /content/data/bpe.vocab.{tgt} --vocabulary-threshold 50 < /content/data/dev/dev.{tgt}.txt > /content/data/dev/dev.{tgt}.bpe

In [ ]:
!python -m sockeye.prepare_data -s /content/data/train/train.{src}.bpe -t /content/data/train/train.{tgt}.bpe --shared-vocab -o /content/{src}_{tgt}_{name}_data

[INFO:sockeye.utils] Sockeye: 3.1.34, commit 4c30942ddb523533bccb4d2cbb3e894e45b1db93, path /usr/local/lib/python3.11/dist-packages/sockeye/__init__.py
[INFO:sockeye.utils] PyTorch: 1.13.1+cu117 (/usr/local/lib/python3.11/dist-packages/torch/__init__.py)
[INFO:sockeye.utils] Command: /usr/local/lib/python3.11/dist-packages/sockeye/prepare_data.py -s /content/data/train/train.rus.bpe -t /content/data/train/train.ckt.bpe --shared-vocab -o /content/rus_ckt_69K_data
[INFO:sockeye.utils] Arguments: Namespace(config=None, source='/content/data/train/train.rus.bpe', source_factors=[], source_factors_use_source_vocab=[], target_factors=[], target_factors_use_target_vocab=[], target='/content/data/train/train.ckt.bpe', end_of_prepending_tag=None, source_vocab=None, target_vocab=None, source_factor_vocabs=[], target_factor_vocabs=[], shared_vocab=True, num_words=(0, 0), word_min_count=(1, 1), pad_vocab_to_multiple_of=8, no_bucketing=False, bucket_width=8, bucket_scaling=False, max_seq_len=(95, 9

In [ ]:
!python -m sockeye.train -d /content/{src}_{tgt}_{name}_data -vs /content/data/dev/dev.{src}.bpe -vt /content/data/dev/dev.{tgt}.bpe --encoder transformer --decoder transformer --transformer-model-size 512 --transformer-feed-forward-num-hidden 2048 --transformer-dropout-prepost 0.1 --num-embed 512 --max-seq-len 100 --decode-and-evaluate 500 -o {src}_{tgt}_{name}_model --num-layers 6 --keep-last-params 3 --batch-size 1024 --optimized-metric bleu --max-num-checkpoint-not-improved 5

[INFO:sockeye.utils] Sockeye: 3.1.34, commit 4c30942ddb523533bccb4d2cbb3e894e45b1db93, path /usr/local/lib/python3.11/dist-packages/sockeye/__init__.py
[INFO:sockeye.utils] PyTorch: 1.13.1+cu117 (/usr/local/lib/python3.11/dist-packages/torch/__init__.py)
[INFO:sockeye.utils] Command: /usr/local/lib/python3.11/dist-packages/sockeye/train.py -d /content/rus_ckt_69K_data -vs /content/data/dev/dev.rus.bpe -vt /content/data/dev/dev.ckt.bpe --encoder transformer --decoder transformer --transformer-model-size 512 --transformer-feed-forward-num-hidden 2048 --transformer-dropout-prepost 0.1 --num-embed 512 --max-seq-len 100 --decode-and-evaluate 500 -o rus_ckt_69K_model --num-layers 6 --keep-last-params 3 --batch-size 1024 --optimized-metric bleu --max-num-checkpoint-not-improved 5
[INFO:sockeye.utils] Arguments: Namespace(config=None, source=None, source_factors=[], source_factors_use_source_vocab=[], target_factors=[], target_factors_use_target_vocab=[], target=None, end_of_prepending_tag=Non

## **Testing the Model**

In [ ]:
!echo "Я люблю бегать." | python3 subword-nmt/subword_nmt/apply_bpe.py -c /content/data/bpe.codes --vocabulary /content/data/bpe.vocab.ckt --vocabulary-threshold 50 | python -m sockeye.translate -m rus_ckt_69K_model 2>/dev/null | sed -r 's/@@( |$)//g'

Мырэмигчирльуӈгъэк.


In [ ]:
!echo "Летом я ловлю рыбу." | python3 subword-nmt/subword_nmt/apply_bpe.py -c /content/data/bpe.codes --vocabulary /content/data/bpe.vocab.ckt --vocabulary-threshold 50 | python -m sockeye.translate -m rus_ckt_69K_model 2>/dev/null | sed -r 's/@@( |$)//g'

Аӈарэгты тырэлеркынигыт.


In [ ]:
!echo "Это мой отец, у него три брата." | python3 subword-nmt/subword_nmt/apply_bpe.py -c /content/data/bpe.codes --vocabulary /content/data/bpe.vocab.ckt --vocabulary-threshold 50 | python -m sockeye.translate -m rus_ckt_69K_model 2>/dev/null | sed -r 's/@@( |$)//g'

Ӄыԓпыгын э'тки ӄытваркынэтык.


In [ ]:
!echo "Прошлой осенью они построили новую ярангу." | python3 subword-nmt/subword_nmt/apply_bpe.py -c /content/data/bpe.codes --vocabulary /content/data/bpe.vocab.ckt --vocabulary-threshold 50 | python -m sockeye.translate -m rus_ckt_69K_model 2>/dev/null | sed -r 's/@@( |$)//g'

Энмэн имтигын тор каԓьатгыргын ынкъам иԓгытэвым.


## **Uploading Model to Hugging Face**

In [ ]:
!pip install transformers huggingface_hub

In [ ]:
from huggingface_hub import notebook_login, HfApi
from transformers import AutoTokenizer
import os

notebook_login()

In [ ]:
# Creating model repository
repo_name = "HSE-Chukchi-NLP/sockeye-rus-ckt-69k"
api = HfApi()
api.create_repo(repo_name, exist_ok=True)

RepoUrl('https://huggingface.co/HSE-Chukchi-NLP/sockeye-rus-ckt-69k', endpoint='https://huggingface.co', repo_type='model', repo_id='HSE-Chukchi-NLP/sockeye-rus-ckt-69k')

In [ ]:
# Saving BPE tokenizer in Hugging Face format
from tokenizers import Tokenizer, models, pre_tokenizers, trainers

with open('/content/data/bpe.codes', 'r', encoding='utf-8') as f:
    bpe_codes = f.read()

In [ ]:
with open('/content/data/bpe.vocab.rus', 'r', encoding='utf-8') as f:
    rus_vocab = [line.split()[0] for line in f.readlines()]

In [ ]:
rus_tokenizer = Tokenizer(models.BPE())
rus_tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Training tokenizer (actually loading existing codes)
trainer = trainers.BpeTrainer(
    vocab_size=10000,
    special_tokens=["<unk>", "<pad>", "<s>", "</s>"]
)

rus_tokenizer.train_from_iterator(
    iterator=rus_vocab,
    trainer=trainer,
    length=len(rus_vocab))
rus_tokenizer.save("/content/rus_tokenizer.json")

In [ ]:
with open('/content/data/bpe.vocab.ckt', 'r', encoding='utf-8') as f:
    ckt_vocab = [line.split()[0] for line in f.readlines()]

ckt_tokenizer = Tokenizer(models.BPE())
ckt_tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
ckt_tokenizer.train_from_iterator(
    iterator=ckt_vocab,
    trainer=trainer,
    length=len(ckt_vocab))
ckt_tokenizer.save("/content/ckt_tokenizer.json")

In [ ]:
# Saving Sockeye model to Hugging Face Hub
!mkdir -p /content/model_files
!cp -r rus_ckt_69K_model /content/model_files
!cp /content/data/bpe.codes /content/model_files/
!cp /content/data/bpe.vocab.ckt /content/model_files/
!cp /content/data/bpe.vocab.rus /content/model_files/

In [ ]:
# Uploading to Hugging Face Hub
api.upload_folder(
    folder_path="/content/model_files",
    repo_id="HSE-Chukchi-NLP/sockeye-rus-ckt-69k",
    repo_type="model"
)

optimizer_best.pkl:   0%|          | 0.00/370M [00:00<?, ?B/s]

params.00029:   0%|          | 0.00/185M [00:00<?, ?B/s]

Upload 15 LFS files:   0%|          | 0/15 [00:00<?, ?it/s]

params.best:   0%|          | 0.00/185M [00:00<?, ?B/s]

lr_scheduler_best.pkl:   0%|          | 0.00/643 [00:00<?, ?B/s]

...

optimizer_last.pkl:   0%|          | 0.00/370M [00:00<?, ?B/s]

params:   0%|          | 0.00/185M [00:00<?, ?B/s]

rng.pkl:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

training.pkl:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HSE-Chukchi-NLP/sockeye-rus-ckt-69k/commit/145120a65125c655b04fdc93af55af5d5045c728', commit_message='Upload folder using huggingface_hub', commit_description='', oid='145120a65125c655b04fdc93af55af5d5045c728', pr_url=None, repo_url=RepoUrl('https://huggingface.co/HSE-Chukchi-NLP/sockeye-rus-ckt-69k', endpoint='https://huggingface.co', repo_type='model', repo_id='HSE-Chukchi-NLP/sockeye-rus-ckt-69k'), pr_revision=None, pr_num=None)

## **Testing on FLORES**

In [ ]:
!pip install datasets sacrebleu

In [ ]:
from datasets import load_dataset
from sacrebleu import corpus_bleu, corpus_chrf
from tqdm import tqdm

# Loading FLORES dataset
flores_dataset = load_dataset("alexantonov/chukot_russian_flores_sample")

# Preparing test data
russian_test = flores_dataset["train"]["ru"]
chukchi_references = [flores_dataset["train"]["ckt"]]

README.md:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/34.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# Function for translating using the model
def translate_texts(texts, bpe_codes_path, vocab_path, model_path):
    translated_texts = []
    for text in tqdm(texts, desc="Translating"):
        # Applying BPE to source text
        !echo "{text}" | python3 subword-nmt/subword_nmt/apply_bpe.py -c {bpe_codes_path} --vocabulary {vocab_path} --vocabulary-threshold 50 > /content/temp_input.bpe

        # Translating
        !python -m sockeye.translate -m {model_path} --input /content/temp_input.bpe --output /content/temp_output.bpe 2>/dev/null

        # Reading result and removing BPE symbols
        with open('/content/temp_output.bpe', 'r', encoding='utf-8') as f:
            translated = f.read().replace('@@ ', '')

        translated_texts.append(translated.strip())

    return translated_texts

# Model parameters
bpe_codes_path = "/content/data/bpe.codes"
vocab_path = "/content/data/bpe.vocab.rus"
model_path = "rus_ckt_69K_model"

In [ ]:
translations = translate_texts(russian_test, bpe_codes_path, vocab_path, model_path)

# Calculating metrics
bleu_score = corpus_bleu(translations, chukchi_references)
chrf_score = corpus_chrf(translations, chukchi_references)

print(f"BLEU score: {bleu_score.score:.2f}")
print(f"ChrF++ score: {chrf_score.score:.2f}")

# Additional information
print("\nBLEU details:")
print(bleu_score)
print("\nChrF++ details:")
print(chrf_score)

Translating:  57%|█████▋    | 57/100 [16:14<12:11, 17.00s/it]

/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


Translating: 100%|██████████| 100/100 [28:25<00:00, 17.05s/it]

BLEU score: 0.20
ChrF++ score: 16.18

BLEU details:
BLEU = 0.20 16.4/0.5/0.0/0.0 (BP = 0.663 ratio = 0.708 hyp_len = 1375 ref_len = 1941)

ChrF++ details:
chrF2 = 16.18


Archive for saving model

In [ ]:
!zip -r res.zip /content/rus_ckt_69K_data /content/data /content/data

  adding: content/rus_ckt_69K_data/ (stored 0%)
  adding: content/rus_ckt_69K_data/shard.00000 (deflated 70%)
  adding: content/rus_ckt_69K_data/log (deflated 72%)
  adding: content/rus_ckt_69K_data/data.info (deflated 40%)
  adding: content/rus_ckt_69K_data/data.version (stored 0%)
  adding: content/rus_ckt_69K_data/args.yaml (deflated 54%)
  adding: content/rus_ckt_69K_data/vocab.trg.0.json (deflated 72%)
  adding: content/rus_ckt_69K_data/vocab.src.0.json (deflated 72%)
  adding: content/rus_ckt_69K_data/data.config (deflated 64%)
  adding: content/data/ (stored 0%)
  adding: content/data/dev/ (stored 0%)
  adding: content/data/dev/dev.rus.txt (deflated 65%)
  adding: content/data/dev/dev.ckt.bpe (deflated 71%)
  adding: content/data/dev/dev.ckt.txt (deflated 66%)
  adding: content/data/dev/dev.rus.bpe (deflated 69%)
  adding: content/data/train/ (stored 0%)
  adding: content/data/train/train.ckt.txt (deflated 69%)
  adding: content/data/train/train.rus.bpe (deflated 72%)
  adding: 